### ***Pydantic***

***Pydentic is the most widely used `data validation library` for Python***

***Pydantic Core validation logic is written in Rust, so it is fastest data validation libraries for Python***

***https://docs.pydantic.dev/latest/***

***You use Pydantic in LangGraph applications primarily to define and validate the state schema of your graph. This ensures that the data flowing through your nodes is type‑safe, structured, and consistent, which is critical when orchestrating complex LLM workflows.***

#### ***How Pydantic fits into LangGraph***

***`State schema definition`: LangGraph applications revolve around a shared state that passes between nodes. Pydantic models let you define this state explicitly with fields, types, and validation rules.***

***`Type validation`: Every update to the graph’s state is checked against the Pydantic schema, catching errors early and preventing malformed data from propagating.***

***`Structured workflows`: By using Pydantic, you can enforce contracts between nodes (e.g., one node must output a summary: str, another must consume it).***

***`Integration with reducers`: LangGraph reducers (functions that update state) can rely on Pydantic models to validate and merge updates safely.***

***`Channel separation`: Developers sometimes subclass LangGraph’s MessageState or use Pydantic models to separate communication channels between agents (e.g., research team vs. writing team).***

#### ***Creating and Using Models***

***Pydantic models are the foundation fo data validation in Python. They use Python type annotations to define the structure and validate data at runtime***

In [2]:
# pip install pydantic

In [4]:
from pydantic import BaseModel

In [5]:
class Person(BaseModel):
    name: str
    age: int
    city: str

person = Person(name="Alice", age=30, city="New York")
print(person)

name='Alice' age=30 city='New York'


In [6]:
type(person)

__main__.Person

In [22]:
# try another example with different fields
try:
    person2 = Person(name="Bob", age=25, city=22) # will give error due to data validation by pydantic
    print(person2)
except Exception as e:
    print(f"Error: {e}")

Error: 1 validation error for Person
city
  Input should be a valid string [type=string_type, input_value=22, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [16]:
# another example without basemodel

class Personality():
    
    def __init__(self, name: str, age: int, city: str):
        self.name = name
        self.age = age
        self.city = city
    
personality = Personality(name="Charlie", age=28, city=22)
print(personality.name)
print(personality.age)
print(type(personality.city))



Charlie
28
<class 'int'>


#### ***2. Model with Optional Field***

***Definition:***

- ***`Optional[type]`: Indicates the field can be `None`***

***Pydantic validates types even for optional fields when values are provided***

In [17]:
from typing import Optional

class Employee(BaseModel):
    name: str
    age: int
    department: str
    salary: Optional[float] = None  # Optional field with default value None
    is_full_time: Optional[bool]  = True  # Optional field with default value True

In [19]:
emp1 = Employee(name="David", age=35, department="HR", salary=60000.0)
print(emp1)

emp2 = Employee(name="Eva", age=29, department="IT")
print(emp2)

name='David' age=35 department='HR' salary=60000.0 is_full_time=True
name='Eva' age=29 department='IT' salary=None is_full_time=True


#### ***Model with Nested Models***

In [23]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    zip_code: str

class User(BaseModel):
    username: str
    email: str
    address: Address

customer = User(
    username="john_doe",
    email="abc@yahoo.com",
    address={
        "street": "123 Main St",
        "city": "Springfield",
        "zip_code": "12345"
    }
)

print(customer)

username='john_doe' email='abc@yahoo.com' address=Address(street='123 Main St', city='Springfield', zip_code='12345')


#### ***Pydantic Fields: Customization adn Constraints***

***The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more.***

In [24]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(min_length=3, max_length=50)  # name must be between 3 and 50 characters
    price: float = Field(..., gt=0)  # price must be greater than 0; dots make it required
    quantity: int = Field(..., ge=0)  # quantity must be greater than or equal to 0: dots make it required

product = Product(name="Laptop", price=999.99, quantity=10)
print(product)

name='Laptop' price=999.99 quantity=10


In [29]:
product.model_json_schema()  # shows the schema of the model

{'properties': {'name': {'maxLength': 50,
   'minLength': 3,
   'title': 'Name',
   'type': 'string'},
  'price': {'exclusiveMinimum': 0, 'title': 'Price', 'type': 'number'},
  'quantity': {'minimum': 0, 'title': 'Quantity', 'type': 'integer'}},
 'required': ['name', 'price', 'quantity'],
 'title': 'Product',
 'type': 'object'}

In [27]:
# simulate error
try:
    product2 = Product(name="TV", price=-500, quantity=5)  # will give error due to validation
    print(product2)
except Exception as e:
    print(f"Error: {e}")

Error: 2 validation errors for Product
name
  String should have at least 3 characters [type=string_too_short, input_value='TV', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short
price
  Input should be greater than 0 [type=greater_than, input_value=-500, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than
